In [12]:
import numpy as np 
from scipy.io import loadmat
import random
from sklearn.metrics import confusion_matrix as cm

#Gaussian activation function
def gaussian(X, a, b):   
    K = np.zeros((X.shape[0], hidden_neurons))
    for i in range(K.shape[0]):
        for j in range(K.shape[1]):
            K[i,j] = np.exp(-b[j] * np.linalg.norm(a[:,j] - X[i,:]))  
    return K

#Tanh activation function
def tanh(X):
    return np.tanh(X)

#Load data, shuffle and normalize
def init_data(data):
    X = np.array(data[ : , :-1], dtype = float)
    y = np.array(data[ : , -1], dtype = int)
    X = (X - X.mean(axis = 0))/X.std(axis = 0)
    return X, y

df = loadmat('data5.mat')
data = df['x']
np.random.shuffle(data)

X_tot, y_tot = init_data(data)
X_tot = np.insert(X_tot, 0, 1, axis=1)

#Generate labels matrix
labels = data[:, -1]
y = np.zeros([len(X_tot), 2])

for i in range(len(labels)):
    if labels[i] == 1:
        y[i,1] = 1.0
    elif labels[i] == 0:
        y[i,0] = 1.0

hidden_neurons = 400
output_neurons = 2

#5 fold cross validn
#Gaussian activation function
print('Gaussian ELM: ')
acc = 0
for k in range(5):

    X_train = X_tot[0 : 1718]
    y_train = y[0 : 1718]
    X_val = X_tot[1718 :]
    y_val = y[1718 :]
    
    a = np.random.rand(X_train.shape[1], hidden_neurons) 
    b = np.random.rand(hidden_neurons)
    
    # Training 
    H = gaussian(X_train, a, b)
    H_inv = np.linalg.pinv(H)
    W2 = np.matmul(H_inv, y_train)
    
    # Testing
    H_T = gaussian(X_val, a, b)
    y_pred = np.matmul(H_T, W2)
    
    print('\nFold ',k+1)
    a = [np.argmax(y_pred[i]) for i in range(len(y_pred))]
    b= [np.argmax(y_val[i]) for i in range(len(y_val))]
    confmat = cm(b,a)
    print('confmat:')
    print(confmat)
    Accuracy = (confmat[0][0]+confmat[1][1])/len(y_val)
    acc += Accuracy
    se = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
    sp = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
    print('Accuracy: ',Accuracy,'sensitivity: ',se,'specificity: ',sp)

    X_tot[0 : 430] = X_val
    X_tot[430 : ] = X_train
    y[0 : 430] = y_val
    y[430 : ] = y_train

print('Avg Accuracy: ',acc/5)

#Best accuracy recorded for fold 1 of gaussian

Gaussian ELM: 

Fold  1
confmat:
[[189  26]
 [ 12 203]]
Accuracy:  0.9116279069767442 sensitivity:  0.9441860465116279 specificity:  0.8790697674418605

Fold  2
confmat:
[[205  13]
 [ 16 196]]
Accuracy:  0.9325581395348838 sensitivity:  0.9245283018867925 specificity:  0.9403669724770642

Fold  3
confmat:
[[194  21]
 [ 14 201]]
Accuracy:  0.9186046511627907 sensitivity:  0.9348837209302325 specificity:  0.9023255813953488

Fold  4
confmat:
[[194  28]
 [ 25 183]]
Accuracy:  0.8767441860465116 sensitivity:  0.8798076923076923 specificity:  0.8738738738738738

Fold  5
confmat:
[[194  20]
 [ 14 202]]
Accuracy:  0.9209302325581395 sensitivity:  0.9351851851851852 specificity:  0.9065420560747663
Avg Accuracy:  0.912093023255814


In [13]:
#Tanh ELM
y_pred_full = []
print("tanh ELM: ")
acc = 0
for k in range(5):
    
    X_train = X_tot[0 : 1718]
    y_train = y[0 : 1718]
    X_val = X_tot[1718 :]
    y_val = y[1718 :]
    
    a = np.random.rand(X_train.shape[1],hidden_neurons) 
    b = np.random.rand(hidden_neurons)
    
    # Training 
    H = tanh(X_train)
    H_inv = np.linalg.pinv(H)
    W2 = np.matmul(H_inv, y_train)
    # Testing
    H_T = tanh(X_val)
    y_pred = np.matmul(H_T, W2)
    

    print('\nFold ',k+1,'\n')
    a = [np.argmax(y_pred[i]) for i in range(len(y_pred))]
    b= [np.argmax(y_val[i]) for i in range(len(y_val))]
    confmat = cm(b,a)
    print("confmat:\n")
    print(confmat)
    Accuracy = (confmat[0][0]+confmat[1][1])/len(y_val)
    acc += Accuracy
    se = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
    sp = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
    print('Accuracy: ',Accuracy,'sensitivity: ',se,'specificity: ',sp)
    
    X_tot[0 : 430] = X_val
    X_tot[430 : ] = X_train
    y[0 : 430] = y_val
    y[430 : ] = y_train
print('Avg Accuracy: ',acc/5)

#Best accuracy recorded for gaussian fold 1 and tanh(comparatively very less) fold 4

tanh ELM: 

Fold  1 

confmat:

[[187  30]
 [ 23 190]]
Accuracy:  0.8767441860465116 sensitivity:  0.892018779342723 specificity:  0.8617511520737328

Fold  2 

confmat:

[[177  39]
 [ 19 195]]
Accuracy:  0.8651162790697674 sensitivity:  0.9112149532710281 specificity:  0.8194444444444444

Fold  3 

confmat:

[[187  36]
 [ 24 183]]
Accuracy:  0.8604651162790697 sensitivity:  0.8840579710144928 specificity:  0.8385650224215246

Fold  4 

confmat:

[[183  30]
 [ 21 196]]
Accuracy:  0.8813953488372093 sensitivity:  0.9032258064516129 specificity:  0.8591549295774648

Fold  5 

confmat:

[[185  32]
 [ 22 191]]
Accuracy:  0.8744186046511628 sensitivity:  0.8967136150234741 specificity:  0.8525345622119815
Avg Accuracy:  0.8716279069767442
